In [1]:
import sys
sys.path.append('hexagon_env')

In [2]:
import functools
import gymnasium
import numpy as np
from gymnasium.spaces import Discrete
from pettingzoo import AECEnv
from pettingzoo.utils import agent_selector, wrappers
import pygame

from hexagon_env.helperDisplay import clearGrid, init_hexagons, initializeAgent, render, renderAgents
from hexagon_env.helperDisplay import predatorDirectionGenerator,preyDirectionGenerator,predator_vision,prey_vision


In [1]:
NO_OF_PREY = 5
NO_OF_PREDATOR = 5
GRID_SIZE = (79,39)

def env(render_mode=None):
    """
    The env function often wraps the environment in wrappers by default.
    You can find full documentation for these methods
    elsewhere in the developer documentation.
    """
    internal_render_mode = render_mode if render_mode != "ansi" else "human"
    env = raw_env(render_mode=internal_render_mode)
    # this wrapper helps error handling for discrete action spaces
    env = wrappers.AssertOutOfBoundsWrapper(env)
    # Provides a wide vareity of helpful user errors
    # Strongly recommended
    env = wrappers.OrderEnforcingWrapper(env)
    return env

class raw_env(AECEnv):
    """
    The metadata holds environment constants. From gymnasium, we inherit the "render_modes",
    metadata which specifies which modes can be put into the render() method.
    At least human mode should be supported.
    The "name" metadata allows the environment to be pretty printed.
    """

    metadata = {"render_modes": ["human"], "name": "hex_v1"}

    def __init__(self, render_mode=None):
        """
        The init method takes in environment arguments and
         should define the following attributes:
        - possible_agents
        - render_mode

        Note: as of v1.18.1, the action_spaces and observation_spaces attributes are deprecated.
        Spaces should be defined in the action_space() and observation_space() methods.
        If these methods are not overridden, spaces will be inferred from self.observation_spaces/action_spaces, raising a warning.

        These attributes should not be changed after initialization.
        """
        self.possible_prey = ["prey_" + str(r) for r in range(NO_OF_PREY)]
        self.possible_predator = ["predator_" + str(r) for r in range(NO_OF_PREDATOR)]
        self.possible_agents = self.possible_prey.copy() + self.possible_predator.copy()

        self.prey_agents = self.possible_prey.copy()
        self.predator_agents = self.possible_predator.copy()

        # optional: a mapping between agent name and ID
        self.agent_name_mapping = dict(
            zip(self.possible_agents, list(range(len(self.possible_agents))))
        )

        # optional: we can define the observation and action spaces here as attributes to be used in their corresponding methods
        self._action_spaces = dict([(agent, Discrete(7)) for agent in self.possible_prey] + [(agent,Discrete(9)) for agent in self.possible_predator])
        self._observation_spaces = dict([(agent, Discrete(36)) for agent in self.possible_prey] + [(agent,Discrete(15)) for agent in self.possible_predator])
        self.predatorAgents,self.preyAgents = initializeAgent(NO_OF_PREDATOR,NO_OF_PREY)

        self.render_mode = render_mode
        if self.render_mode == "human":
            pygame.init()
            pygame.font.init()
            self.clock = pygame.time.Clock()
            self.world = pygame.display.set_mode((1200, 700))
            self.hexagons = init_hexagons(flat_top=True)
            self.clock.tick(70)
            # clearGrid(self.hexagons)
            # renderAgents(self.preyAgents,self.predatorAgents,self.hexagons)
            # render(self.world, self.hexagons)


    # Observation space should be defined here.
    # lru_cache allows observation and action spaces to be memoized, reducing clock cycles required to get each agent's space.
    # If your spaces change over time, remove this line (disable caching).
    @functools.lru_cache(maxsize=None)
    def observation_space(self, agent):
        # gymnasium spaces are defined and documented here: https://gymnasium.farama.org/api/spaces/
        if agent in self.possible_predator:
            return Discrete(15)
        else:
            return Discrete(36)
    
    # Action space should be defined here.
    # If your spaces change over time, remove this line (disable caching).
    @functools.lru_cache(maxsize=None)
    def action_space(self, agent):
        if agent in self.possible_predator:
            return Discrete(9)
        else:
            return Discrete(7)

    def render(self):
        """
        Renders the environment. In human mode, it can print to terminal, open
        up a graphical window, or open up some other display that a human can see and understand.
        """
        if self.render_mode is None:
            print('no agent specified')
            return
        
        if len(self.prey_agents) > 0 and len(self.predator_agents) > 0:
            clearGrid(self.hexagons)
            renderAgents(self.preyAgents,self.predatorAgents,self.hexagons)
            render(self.world, self.hexagons)

        else:
            string = "Game over"
            print(string)
            self.close()


    def observe(self, agent):
        """
        Observe should return the observation of the specified agent. This function
        should return a sane observation (though not necessarily the most up to date possible)
        at any time after reset() is called.
        """
        # observation of one agent is the previous state of the other
        return np.array(self.observations[agent])

    def close(self):
        """
        Close should release any graphical displays, subprocesses, network connections
        or any other environment data which should not be kept around after the
        user is no longer using the environment.
        """
        pygame.display.quit()


    def reset(self, seed=None, options=None):
        """
        Reset needs to initialize the following attributes
        - agents
        - rewards
        - _cumulative_rewards
        - terminations
        - truncations
        - infos
        - agent_selection
        And must set up the environment so that render(), step(), and observe()
        can be called without issues.
        Here it sets up the state dictionary which is used by step() and the observations dictionary which is used by step() and observe()
        """
        self.agents = self.possible_agent.copy()
        self.rewards = {agent: 0 for agent in self.agents}
        self._cumulative_rewards = {agent: 0 for agent in self.agents}
        self.terminations = {agent: False for agent in self.agents}
        self.truncations = {agent: False for agent in self.agents}
        self.infos = {agent: {} for agent in self.agents}
        self.state = {agent: None for agent in self.agents}
        # self.observations = {agent: [0]*36 for agent in self.agents}
        self.observations = dict([(agent,[0]*36)  for agent in self.prey_agents]+[(agent,[0]*15)  for agent in self.predator_agents])
        self.num_moves = 0
        self.prey_agents = self.possible_prey.copy()
        self.predator_agents = self.possible_predator.copy()

        """
        Our agent_selector utility allows easy cyclic stepping through the agents list.
        """
        self._agent_selector = agent_selector(self.agents)
        self.agent_selection = self._agent_selector.next()

    def step(self, action):
        """
        step(action) takes in an action for the current agent (specified by
        agent_selection) and needs to update
        - rewards
        - _cumulative_rewards (accumulating the rewards)
        - terminations
        - truncations
        - infos
        - agent_selection (to the next agent)
        And any internal state used by observe() or render()
        """
        
                
        agent = self.agent_selection

        if (
            self.truncations[agent]
        ):
            # handles stepping an agent which is already dead
            # accepts a None action for the one agent, and moves the agent_selection to
            # the next dead agent,  or if there are no more dead agents, to the next live agent
            self.close()
            return

        preyTrue = False

        # check if prey or predator and store index
        if agent in self.prey_agents:
            index = self.prey_agents.index(agent)
            preyTrue = True
        else:
            index = self.predator_agents.index(agent)
            preyTrue = False
        
        if (self.terminations[agent]):
            # handles stepping an agent which is already dead
            # accepts a None action for the one agent, and moves the agent_selection to
            # the next dead agent, or if there are no more dead agents, to the next live agent
            if preyTrue:
                del self.preyAgents[index]     # data
                del self.prey_agents[index]    # namelist
            else:
                del self.predatorAgents[index]
                del self.predator_agents
            self._was_dead_step(action)
            return
        
        if preyTrue:
            # if prey and action .. is taken move accordinly
            x,y = preyDirectionGenerator(self.preyAgents[index][0],self.preyAgents[index][1],action)
            self.preyAgents[index][0] = x
            self.preyAgents[index][1] = y           
        else:
            # if predator and action .. is taken then move or change dir accordingly
            x,y,dir = predatorDirectionGenerator(self.predatorAgents[index][0],self.predatorAgents[index][1],self.predatorAgents[index][4],action)
            self.predatorAgents[index][0] = x
            self.predatorAgents[index][1] = y           
            self.predatorAgents[index][4] = dir

        # the agent which stepped last had its _cumulative_rewards accounted for
        # (because it was returned by last()), so the _cumulative_rewards for this
        # agent should start again at 0
        # self._cumulative_rewards[agent] = 0
        
        # stores action of current agent
        # self.state[self.agent_selection] = action

        # collect reward if it is the last agent to act
        if self._agent_selector.is_last():
            # rewards for all agents are placed in the .rewards dictionary

            self.num_moves += 1
            # The truncations dictionary must be updated for all players.
            # self.truncations = {
            #     agent: self.num_moves >= NUM_ITERS for agent in self.agents
            # }

            # observe the current state
            # for i in self.agents:
            #     self.observations[i] = self.state[
            #         self.agents[1 - self.agent_name_mapping[i]]
            #     ]
            for i in self.predatorAgents:
                # returns the observation for all alive predator agents
                visionList = predator_vision((i[0],i[1]),i[4])

                for visionIndex,visionhex in enumerate(visionList):
                    index = self.predatorAgents.index(i)

                    for prey in self.preyAgents:
                        temp_obs = self.observations[self.predator_agents[index]]
                        if (prey[0],prey[1]) == visionhex:
                            temp_obs[visionIndex] = 1
                            self.observations[self.predator_agents[index]] = temp_obs
                            break
                        else:
                            temp_obs[visionIndex] = 0
                            self.observations[self.predator_agents[index]] = temp_obs


            for i in self.preyAgents:
                # returns the observation for all alive prey agents
                visionList = prey_vision((i[0],i[1]),3)
                for visionIndex,visionhex in enumerate(visionList):
                    index = self.preyAgents.index(i)
                    
                    for predator in self.predatorAgents:
                        temp_obs = self.observations[self.prey_agents[index]]
                        if (predator[0],predator[1]) == visionhex:
                            temp_obs[visionIndex] = 1
                            self.observations[self.prey_agents[index]] = temp_obs
                            break
                        else:
                            temp_obs[visionIndex] = 0
                            self.observations[self.prey_agents[index]] = temp_obs
            

        else:
            # necessary so that observe() returns a reasonable observation at all times.
            # self.state[self.agents[1 - self.agent_name_mapping[agent]]] = None
            # no rewards are allocated until both players give an action
            self._clear_rewards()

        # selects the next agent.
        self.agent_selection = self._agent_selector.next()
        # Adds .rewards to ._cumulative_rewards
        self._accumulate_rewards()

        if self.render_mode == "human":
            self.render()

NameError: name 'AECEnv' is not defined

In [ ]:
import time
import random
a = env = raw_env(render_mode="human")
a.reset()
count = 10000

for i in range(count):
    for event in pygame.event.get():
            if event.type == pygame.QUIT:
                terminated = True
    # actions = []
    # for agent in a.possible_agents:
    #     actions.append((agent,1))
    # actions = dict(actions)
    a.step(random.randint(2,3))
    # a.render()


[(15, -3), (15, -4), (15, -5), (16, -3), (16, -4), (16, -5), (17, -3), (17, -4), (17, -5)]
[(67, 5), (68, 5), (69, 5), (66, 6), (67, 6), (68, 6), (65, 7), (66, 7), (67, 7)]
[(70, -33), (71, -34), (72, -35), (70, -32), (71, -33), (72, -34), (70, -31), (71, -32), (72, -33)]
[(40, 8), (41, 8), (42, 8), (39, 9), (40, 9), (41, 9), (38, 10), (39, 10), (40, 10)]
[(66, -4), (65, -4), (64, -4), (67, -5), (66, -5), (65, -5), (68, -6), (67, -6), (66, -6)]
[(15, -3), (15, -4), (15, -5), (16, -3), (16, -4), (16, -5), (17, -3), (17, -4), (17, -5)]
[(67, 5), (68, 5), (69, 5), (66, 6), (67, 6), (68, 6), (65, 7), (66, 7), (67, 7)]
[(70, -33), (71, -34), (72, -35), (70, -32), (71, -33), (72, -34), (70, -31), (71, -32), (72, -33)]
[(40, 8), (41, 8), (42, 8), (39, 9), (40, 9), (41, 9), (38, 10), (39, 10), (40, 10)]
[(66, -4), (65, -4), (64, -4), (67, -5), (66, -5), (65, -5), (68, -6), (67, -6), (66, -6)]
[(15, -3), (15, -4), (15, -5), (16, -3), (16, -4), (16, -5), (17, -3), (17, -4), (17, -5)]
[(67, 5), (

KeyboardInterrupt: 